<a href="https://colab.research.google.com/github/jamiehepburn/Honours-Project/blob/SAMI-Kinematic-Maps/5_fold_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

In [ ]:

# Model configuration
batch_size = 538
img_width, img_height, img_num_channels = 50, 50, 1
loss_function = binary_crossentropy
no_classes = 3
no_epochs = 300
optimizer = Adam()
verbosity = 0
num_folds = 5

# Load CIFAR-10 data
# setup data into obvious vs non obvious rotation

X = np.load('/content/drive/MyDrive/KIN_TRAINING_DATASETS/HUMAN_NORMALIZED/X.npy')
y = np.load('/content/drive/MyDrive/KIN_TRAINING_DATASETS/HUMAN_NORMALIZED/y.npy')

y = np.where(y > 1, 1, 0)

input_train = X
input_test = y

input_train = np.reshape(input_train, (-1,50,50,1))

# remove all categories greater than KIN_MTYPE 3 (face on spirals and unclassified)
# y = y[np.where(y <= 5)]
# X = X[np.where(y <= 5)]

# Determine shape of the data
input_shape = (img_width, img_height, img_num_channels)

# Parse numbers as floats
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Normalize data
# input_train = input_train / 255
# input_test = input_test / 255

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = input_train
targets = input_test

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(50, 50, 1), kernel_regularizer= tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.AveragePooling2D(2, 2),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.AveragePooling2D(2, 2),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu',kernel_regularizer= tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: loss of 0.31569916009902954; accuracy of 87.96296119689941%
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: loss of 0.3461098372936249; accuracy of 85.18518805503845%
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: loss of 0.3695586025714874; accuracy of 85.18518805503845%
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: loss of 0.3445814847946167; accuracy of 82.2429895401001%
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: loss of 0.24897827208042145; accuracy of 90.65420627593994%
------------------------------------------------------------------------
Score per fold
-----------------------------------------